<a href="https://www.kaggle.com/code/mikedelong/what-year-s-were-the-most-presidents-alive?scriptVersionId=163241441" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

This seems like a straightforward question, and as we see below it is easy to visualize the lifetimes of US Presidents using plotly. 

In [1]:
import numpy as np
import pandas as pd
from arrow import Arrow
from arrow import now

def make_date(arg, year, month, day):
    # some presidents are alive now so we pretend for purposes of this analysis that you know what
    if np.isnan(arg[year]):
        return now(tz=None).date()
    return Arrow(year=int(arg[year]), month=int(arg[month]), day=int(arg[day]), ).date()

PRESIDENTS = '/kaggle/input/historical-us-president-physical-data-more/Historical Presidents Physical Data (More).csv'
df = pd.read_csv(filepath_or_buffer=PRESIDENTS, )
df['birth'] = df.apply(func=make_date, axis=1, args=('birth_year', 'birth_month', 'birth_day',))
df['death'] = df.apply(func=make_date, axis=1, args=('death_year', 'death_month', 'death_day',))
df = df[['name', 'birth', 'death']]
df.head()

,name,birth,death
0,George Washington,1732-02-22,1799-12-14
1,John Adams,1735-10-30,1826-07-04
2,Thomas Jefferson,1743-04-13,1826-07-04
3,James Madison,1751-03-16,1836-06-28
4,James Monroe,1758-04-28,1831-07-04


In [2]:
from plotly.express import timeline
timeline(data_frame=df, x_start='birth', x_end='death', y='name', height=800)

Careful observers will note we are considering Grover Cleveland to be one President instead of two for the sake of answering this question.

We have the problem formulated in people/dates terms, but we have asked a year-centric problem. Let's transform our data to get something in people/years.

In [3]:
def get_years(arg) -> list():
    start = arg['birth'].year
    end = arg['death'].year
    return list(range(start,end+1))

people = []
for index, row in df.iterrows():
    years = get_years(row)
    result = [year in years for year in range(1732,now().date().year+1)]
    result = pd.Series(data=result, index=range(1732,now().date().year+1))
    people.append(result)
result_df = pd.DataFrame(data=people, index=df['name'].values)
result_df.head()

,1732,1733,1734,1735,1736,1737,1738,1739,1740,1741,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
George Washington,True,True,True,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
John Adams,False,False,False,True,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
Thomas Jefferson,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
James Madison,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
James Monroe,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Now we have something we can sum to get the number of US Presidents alive that year.

In [4]:
print(result_df.sum().max())

18


And we know that the year(s) with the most living Presidents will have 18 living Presidents, which seems like a lot. Let's find the years.

In [5]:
final_df = result_df.sum().to_frame().reset_index()
final_df[final_df[0] == 18].T

,90,91,92,93,94,99,101,102,103,104,105,106,107,108,109,111,112,113
index,1822,1823,1824,1825,1826,1831,1833,1834,1835,1836,1837,1838,1839,1840,1841,1843,1844,1845
0,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18,18


Wow that seems like a lot. Let's visualize.

In [6]:
from plotly.express import histogram
histogram(data_frame=result_df.sum().to_frame().reset_index(), x='index', y=0, nbins=now().date().year+1-1732)

Sure enough. Who were the 18 Presidents alive in 1845?

In [7]:
who_df = result_df[[1845]]
who_df = who_df[who_df[1845]]
who_df.index.tolist()

['John Quincy Adams',
 'Andrew Jackson',
 'Martin Van Buren',
 'John Tyler',
 'James Knox Polk',
 'Zachary Taylor',
 'Millard Fillmore',
 'Franklin Pierce',
 'James Buchanan',
 'Abraham Lincoln',
 'Andrew Johnson',
 'Ulysses Simpson Grant',
 'Rutherford Birchard Hayes',
 'James Abram Garfield',
 'Chester Alan Arthur',
 'Stephen Grover Cleveland',
 'Benjamin Harrison',
 'William McKinley']

This makes sense; the run-up to the Civil War included a lot of one-term Presidents, which would mean a lot of Presidents living at the same time. In this list we have several who served less than one full term, including three who were assasinated: Lincoln, Garfield, and McKinley. Yikes!